In [2]:
!pip install confluent_kafka


In [4]:
import json
import random
import time
from confluent_kafka import Producer

# List of vehicle types and colors
vehicle_types = ["motorcycle", "bus", "car", "bicycle", "truck"]
colors = ["red", "blue", "green", "yellow", "white"]
speed_ranges = {
    "motorcycle": (60, 150),
    "bus": (40, 100),
    "car": (80, 150),
    "bicycle": (10, 30),  # Fixed the typo in "cycle" to "bicycle"
    "truck": (60, 120)
}
highways = ["N1", "N2", "N3", "N4", "N5"]
directions = ["northbound", "southbound", "eastbound", "westbound"]

# Kafka producer configuration
producer_config = {
    'bootstrap.servers': 'localhost:9091',  # Replace with your Kafka broker address
    'client.id': 'traffic-producer'
}

# Kafka topic to send traffic data to
topic = 'traffic_data'

# Function to generate random traffic data
def generate_traffic_data():
    producer = Producer(producer_config)
    start_time = time.time()
    
    for _ in range(5):  # Run for (n)seconds
        vehicle = {
            "highway": random.choice(highways),
            "type": random.choice(vehicle_types),
            "color": random.choice(colors),
            "direction": random.choice(directions),
            "speed": random.randint(40, 120),  # Speed in km/h
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        try:
            # Send the data to the Kafka topic
            producer.produce(topic, key=None, value=json.dumps(vehicle))
            print("Data sent successfully")
        except Exception as e:
            print("Error sending data:", str(e))
        
        time.sleep(1)  # Generate data every second



In [5]:
generate_traffic_data()
producer.flush()
time.sleep(5)


Data sent successfully
Data sent successfully
Data sent successfully
Data sent successfully
Data sent successfully


%4|1693134832.995|TERMINATE|traffic-producer#producer-3| [thrd:app]: Producer terminating with 5 messages (636 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


NameError: name 'producer' is not defined

In [8]:
from confluent_kafka import Producer

# Kafka producer configuration
producer_config = {
    'bootstrap.servers': 'localhost:9091',  # Replace with your Kafka broker address
    'client.id': 'traffic-producer'
}

# Kafka topic to send traffic data to
topic = 'traffic_data'

# Create a Kafka producer instance
producer = Producer(producer_config)

# Function to generate random traffic data
def generate_traffic_data():
    while True:
        vehicle = {
            "highway": random.choice(highways),
            "type": random.choice(vehicle_types),
            "color": random.choice(colors),
            "direction": random.choice(directions),
            "speed": random.randint(*speed_ranges.get("car")),  # Adjust based on the chosen vehicle type
            "timestamp": int(time.time())
        }
        yield vehicle
        time.sleep(1)  # Generate data every second

if __name__ == "__main__":
    data_generator = generate_traffic_data()

    try:
        for _ in range(3):  # Generate data for 5 minutes (300 seconds)
            vehicle_data = next(data_generator)
            key = "vehicle_" + str(time.time())  # Use a unique key for each message

            # Send the data to the Kafka topic
            producer.produce(topic, key=key, value=json.dumps(vehicle_data))
            print("Data sent successfully")

            time.sleep(1)  # Generate data every second
    except Exception as e:
        print("Error sending data:", str(e))
    finally:
        producer.flush()


Data sent successfully
Data sent successfully
Data sent successfully


%6|1693135821.534|FAIL|traffic-producer#producer-6| [thrd:localhost:9091/bootstrap]: localhost:9091/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 1ms in state APIVERSION_QUERY)
%6|1693135821.535|FAIL|traffic-producer#producer-6| [thrd:127.0.0.1:9091/1]: 127.0.0.1:9091/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 1ms in state APIVERSION_QUERY)
%6|1693135821.619|FAIL|traffic-producer#producer-6| [thrd:localhost:9091/bootstrap]: localhost:9091/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 1 